## Part 1a: Acquiring and returning resources
What is the most obvious issue? How to fix it? (without context managers)

In [6]:

files=[]
for x in range(10000):
    #print(x)
    files.append(open('output.txt', 'w'))

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3267, in run_code
  File "<ipython-input-6-85c24bd75fd3>", line 5, in <module>
    files.append(open('output.txt', 'w'))
OSError: [Errno 24] Too many open files: 'output.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2018, in showtraceback
AttributeError: 'OSError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 1095, in get_records
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
  File "/anaconda3/lib/python3.7/site-packages/IPython/core/ultratb.py", line 347, in _fixed_getinnerframes
  File "/an

OSError: [Errno 24] Too many open files: 'output.txt'

In [2]:
len(files)

199

The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.


In [5]:
# Determine which files are open in a list
def check_closed(list_files):
    print("There are {} files in the list".format(len(list_files)))
    for i in range(len(list_files)):
        if not list_files[i].closed:
            print('File not closed: {}'.format(i))
    print("Done!")

In [11]:
check_closed(files)

There are 10000 files in the list
Done!


## Part 1b

In [12]:
files=[]
for x in range(10):
    f = open('foo.txt', 'w')
    files.append(f)
    if x==5: raise Exception('exception raised')
    f.close()

Exception: exception raised

In [13]:
check_closed(files)

There are 6 files in the list
File not closed: 5
Done!


## Part 2: long computation
- What is a possible solution with context manager?
  - Help me use the "with" statement. Why might this be better?

In [14]:
## delays things for 5 seconds
from timeit import timeit
files = []

def sum_squares(n_squares=6):
    squared = []
    for i in range(n_squares):
        squared.append(i**2)
    summed_value = 0
    for i in squared:
        summed_value += i
    return summed_value


def write_value_to_file():
    f = open('output.txt', 'w')
    files.append(f)
    time_it_took = timeit(sum_squares)
    msg = 'The time to compute is {} seconds\n'.format(time_it_took)
    print(msg)
    f.write(msg)
    f.close()

write_value_to_file() 
check_closed(files)


The time to compute is 2.220833342999981 seconds

There are 1 files in the list
Done!


## Part 3: Constructing our own Context Manager to do the same thing
Q: What is a context manager?

What to do if I comment out the exception?

In [20]:
'''
Part 3: Constructing our own Context Manager to do the same thing
http://book.pythontips.com/en/latest/context_managers.html#implementing-a-context-manager-as-a-class
'''
files = []

class File(object):
    """Starter context manager
    """
    def __init__(self, file_name, method):
        print('__init__')
        self.file_obj = open(file_name, method)
    def __enter__(self):
        print('enter')
        return self.file_obj
    def __exit__(self, type, value, traceback):
        print('exit')
        self.file_obj.close()

item_test=File('output.txt', 'w')


__init__


In [21]:
with item_test as f:
    files.append(f)
    time_it_took = timeit(sum_squares) # 5 seconds after enter
    msg = 'The time to compute is {} seconds\n'.format(time_it_took)
    #raise Exception('exception')
    print(msg)
    f.write(msg)
check_closed(files)


enter
The time to compute is 2.199377118999905 seconds

exit
There are 1 files in the list
Done!


## Part 4: contextlib

In [22]:
'''
Part 4: Contextlib contextmanager
https://docs.python.org/3/library/contextlib.html#contextlib.contextmanager
'''

from contextlib import contextmanager

@contextmanager
def open_file(path, mode):
    the_file = open(path, mode)
    yield the_file
    the_file.close()

files = []

for x in range(10):
    with open_file('foo.txt', 'w') as infile:
        files.append(infile)

check_closed(files)


There are 10 files in the list
Done!
